In [1]:
import deeplake
import matplotlib.pyplot as plt
import os
import cv2, numpy, vptree, PIL
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms, models
import time
import tensorflow as tf

/home/aaryan/miniconda3/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.0) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
2024-04-12 21:04:17.745265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 21:04:18.771129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def plot_image(img):
    if (len(img.shape) == 2):
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img, interpolation='nearest')
    plt.axis('off')
    plt.show()

In [3]:
def plot_two(image1, image2):
    # Create a figure and axis objects
    fig, axes = plt.subplots(1, 2)
    
    # Plot the first image on the left axis
    axes[0].imshow(image1, cmap='gray')

    axes[0].set_title('Image 1')
    
    # Plot the second image on the right axis
    axes[1].imshow(image2, cmap='gray')
    axes[1].set_title('Image 2')
    
    # Hide axis ticks and labels
    for ax in axes:
        ax.axis('off')
    
    # Show the plot
    plt.show()


In [30]:
ds = deeplake.load('hub://activeloop/wiki-art', 'eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpZCI6ImFhcnlhbnJhbXBhbCIsImFwaV9rZXkiOiJuYW10YWJ3WVVvWnFmWUNPb01nZmNQcDZZTXdqSTREWUNCTlY1dFBaRDNFb0wifQ.')
labels_list = ds.labels.info['class_names']

-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/wiki-art



\

hub://activeloop/wiki-art loaded successfully.



### Defines the transform for each batch

In [45]:
labels_list

['abstract_expressionism',
 'action_painting',
 'analytical_cubism',
 'art_nouveau_modern',
 'baroque',
 'color_field_painting',
 'contemporary_realism',
 'cubism',
 'early_renaissance',
 'expressionism',
 'fauvism',
 'high_renaissance',
 'impressionism',
 'mannerism_late_renaissance',
 'minimalism',
 'naive_art_primitivism',
 'new_realism',
 'northern_renaissance',
 'pointillism',
 'pop_art',
 'post_impressionism',
 'realism',
 'rococo',
 'romanticism',
 'symbolism',
 'synthetic_cubism',
 'ukiyo_e']

In [65]:
filter = ['abstract_expressionism', 'action_painting', 'color_field_painting', 'impressionism', 'minimalism', 'pointillism', 'rococo', 'synthetic_cubism']

In [66]:
filter

['abstract_expressionism',
 'action_painting',
 'color_field_painting',
 'impressionism',
 'minimalism',
 'pointillism',
 'rococo',
 'synthetic_cubism']

In [74]:
@deeplake.compute
def filter_labels(sample_in, labels_list):
    return sample_in.labels.data()['text'] not in labels_list

In [75]:
filtered_ds = ds.filter(lambda sample_in: filter_labels(sample_in, filter))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81433/81433 [00:02<00:00, 27545.43it/s]


In [84]:
filtered_ds.labels.info['class_names']

['abstract_expressionism',
 'action_painting',
 'analytical_cubism',
 'art_nouveau_modern',
 'baroque',
 'color_field_painting',
 'contemporary_realism',
 'cubism',
 'early_renaissance',
 'expressionism',
 'fauvism',
 'high_renaissance',
 'impressionism',
 'mannerism_late_renaissance',
 'minimalism',
 'naive_art_primitivism',
 'new_realism',
 'northern_renaissance',
 'pointillism',
 'pop_art',
 'post_impressionism',
 'realism',
 'rococo',
 'romanticism',
 'symbolism',
 'synthetic_cubism',
 'ukiyo_e']

In [8]:
img_size = 128

tform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([img_size, img_size])
    # transforms.Normalize([0,0,0], [255,255,255])
])

def transform(sample_in):
    # sample_in['images'] is of type numpy.ndarray
    return {'images': tform(sample_in['images']), 'labels': sample_in['labels']}

Create the dataloader

In [9]:
dataloader = ds.pytorch(num_workers=0, batch_size=64, shuffle=True, transform=transform)

In [1]:
# ds.visualize()

Tensor(key='labels')

`process_batch_loop` does most of the transformations on numpy, but then loops through each numpy array (which is an img) and sends it to the edge detector.

`process_batch` does all of the transformations on numpy. Then, it makes the `[x, 200, 200, 3]` numpy array into a `[200 * x, 200, 3]` numpy array, detects edges and splits it back into a `[x, 200, 200, 3]` numpy array

`process_batch_t` does all of the transformations on tensors. Then, it makes the `[x, 200, 200, 3]` tensor into a numpy array before passing it into `detectEdges()` and splits it back into a `[x, 200, 200, 3]` tensor. It then adds all the tensors to `edge_list` and returns the list.

`process_batch_t_stack` is the same as `process_batch_t` but it returns the `[x, 200, 200, 3]` tensor instead of a list.

In [38]:
# query_string = """
# SELECT * 
# FROM ds 
# GROUP BY labels 
# LIMIT 5
# """


In [39]:
# # Define the destination path for the copied dataset
# destination_path = "test/"

# # Copy the dataset to the local destination
# copied_ds = ds.copy(dest=destination_path)

# # Define the query string to perform on the copied dataset
# query_string = """
# SELECT * 
# FROM copied_ds 
# GROUP BY labels 
# LIMIT 5
# """

# # Query the copied dataset
# query_result_ds = copied_ds.query(query_string, runtime={'tensor_db': True}, return_data=True)

# # Print some information about the query result
# print("Number of samples in query result:", len(query_result_ds))


In [40]:
# len(query_result_ds)

In [41]:
def process_batch(batch, edge_detector):
    # print(type(batch))
    batch_numpy = batch['images'].numpy()
    # print(batch_numpy.shape)
    
    transposed_batch = np.transpose(batch_numpy, (0, 2, 3, 1))
    # print(transposed_batch.shape)

    
    combined_image = np.concatenate(transposed_batch, axis=1)
    # plot_image(combined_image)

    # print(type(combined_image), combined_image.shape)

    edges = edge_detector.detectEdges(combined_image)
    # plot_image(edges)

    return np.split(edges, len(batch['images']), axis=1)

In [42]:
def process_batch_loop(batch, edge_detector):
    batch_numpy = batch['images'].numpy()
    transposed_batch = np.transpose(batch_numpy, (0, 2, 3, 1))
    
    edge_list = []
    for img in transposed_batch:
        edges = edge_detector.detectEdges(img)
        edge_list.append(edges)

    return edge_list

In [9]:
def process_batch_t(batch, edge_detector):
    batch_numpy = batch['images']
    # print(batch_numpy.shape)
    
    transposed_batch = tf.transpose(batch_numpy, (0, 2, 3, 1))
    # print(transposed_batch.shape)
    # print(len(batch['images']))
    
    combined_image = tf.reshape(transposed_batch, (img_size * len(batch['images']), img_size, 3)).numpy()
    # print(type(combined_image), combined_image.shape)
    # plot_image(combined_image)

    edges = edge_detector.detectEdges(combined_image)
    # plot_image(edges)

    return tf.split(tf.convert_to_tensor(edges), len(batch['images']), axis=0)

In [10]:
def process_batch_t_stack(batch, edge_detector):
    batch_numpy = batch['images']
    # print(batch_numpy.shape)
    
    transposed_batch = tf.transpose(batch_numpy, (0, 2, 3, 1))
    # print(transposed_batch.shape)
    # print(len(batch['images']))
    
    combined_image = tf.reshape(transposed_batch, (img_size * len(batch['images']), img_size, 3)).numpy()
    # print(type(combined_image), combined_image.shape)
    # plot_image(combined_image)

    edges = edge_detector.detectEdges(combined_image)

    return tf.stack(tf.split(tf.convert_to_tensor(edges), len(batch['images']), axis=0))

### Iterating over the dataloader

In [11]:
edge_detector = cv2.ximgproc.createStructuredEdgeDetection('model.yml')

In [12]:
edge_tensor = None
i = 0
og_list = []

start_time_1 = time.time()
for batch in iter(dataloader):
    og_list.extend(batch['images'])

    obj = process_batch_t_stack(batch, edge_detector)
    
    if edge_tensor is None:
        edge_tensor = obj
    else:
        edge_tensor = tf.concat([edge_tensor, obj], axis = 0)
    i += 1
    if (i == 5):
        end_time_1 = time.time()
        break

2024-04-10 20:33:00.580318: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 20:33:00.581474: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 20:33:00.581613: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
# print(times)
print(edge_tensor.shape[0] / (end_time_1 - start_time_1))

1.3083808628939888


In [2]:
# i = 0
# for tens in edge_tensor:
#     # print(tens.shape)
#     # print(og_list[i].shape)
#     plot_two(tens.numpy(), tf.transpose(og_list[i], [1, 2, 0]).numpy())
#     # plot_image(tens.numpy())
#     i += 1

In [15]:
# print(type(og_list))
og_tensor = tf.stack(og_list)
# print(og_tensor.shape)
# print(og_tensor.shape)
# print(edge_tensor.shape)
torch.save(edge_tensor, 'edges.pt')
torch.save(og_tensor, 'og.pt')

In [3]:
# for i in range(len(edge_tensor)):
#     plot_two(edge_tensor[i], tf.transpose(og_tensor[i], [1, 2, 0]).numpy())

In [17]:
# start = time.time()
saved_batch = torch.load('batch_data.pt')
# print(time.time() - start)
# saved_batch
n = len(saved_batch['images'])

In [18]:
# print(saved_batch['images'])
transposed = tf.transpose(saved_batch['images'], (0, 2, 3, 1))
combined_image = tf.reshape(transposed, (200 * n, 200, 3)).numpy()

In [19]:
edges = edge_detector.detectEdges(combined_image)
tens = tf.stack(tf.split(tf.convert_to_tensor(edges), n, axis = 0))

In [20]:
print(tens.shape)

(64, 200, 200)


In [4]:
# for tensor in tens:
#     plot_image(tensor.numpy())

In [22]:
edge_detector = cv2.ximgproc.createStructuredEdgeDetection('model.yml')

for siz in range(6, 10):
    dataloader = ds.pytorch(num_workers=2, batch_size=2 ** siz, shuffle=False, transform=transform)
    # edge_list_1 = []
    # i = 0
    # start_time_1 = time.time()
    # for batch in iter(dataloader):
    #     edge_list_1.extend(process_batch(batch, edge_detector))
    #     i += 1
    #     if (i == 10):
    #         end_time_1 = time.time()
    #         break
            
    # edge_list_2 = []
    # i = 0
    # start_time_2 = time.time()
    # for batch in iter(dataloader):
    #     edge_list_2.extend(process_batch_loop(batch, edge_detector))
    #     i += 1
    #     if (i == 10):
    #         end_time_2 = time.time()
    #         break

    edge_list_3 = []
    i = 0
    start_time_3 = time.time()
    for batch in iter(dataloader):
        edge_list_3.extend(process_batch_t(batch, edge_detector))
        i += 1
        if (i == 5):
            end_time_3 = time.time()
            break

    # print(f"n = {siz}: naive = {len(edge_list_2)/(end_time_2 - start_time_2)} img/sec, smart = {len(edge_list_1)/(end_time_1 - start_time_1)} img/sec, smart_tensor = {len(edge_list_3)/(end_time_3 - start_time_3)} img/sec")
    print(f"n = {2 ** siz}: smart_tensor = {len(edge_list_3)/(end_time_3 - start_time_3)} img/sec")

/home/aaryan/miniconda3/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


n = 64: smart_tensor = 4.017817847992014 img/sec
n = 128: smart_tensor = 19.789941884455615 img/sec
n = 256: smart_tensor = 14.716899589092263 img/sec
n = 512: smart_tensor = 17.756189379732167 img/sec


In [23]:
im1 = ds.images[27272].numpy() # Fetch the first image and return a numpy array
print(im1.shape)

(482, 604, 3)


In [7]:
# image = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
# image = image.astype(np.float32) / 255.0
# plot_image(image)

In [25]:
triangle = cv2.imread('triangle.jpg')
# triangle = cv2.cvtColor(triangle, cv2.COLOR_GRAY2BGR)
triangle_color = cv2.merge([triangle])
triangle = triangle_color.astype(np.float32) / 255.0
# plot_image(triangle)
print(triangle.shape)

(533, 600, 3)


`Canny()` requires a grayscale image. It is also very senstitive to outliers so you add a blur function to remove some of the outliers. The blurring is not necessary but helps to reduce unnecessary edges.

In [26]:
gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred,100,200)

In [6]:
# # plot_image(gray)
# # plot_image(blurred)
# plot_image(edges)

`detectEdges()` requires a color image, and it wants each value of the array to be a float between 0 and 1. Therefore, you convert it to a float, and divide it by 255 (the maximum value).

In [28]:
image = im1.astype(np.float32) / 255.0
# print(image[0])
print(image.shape)
# detectEdges requires a color image
# print(image)

edges = edge_detector.detectEdges(image)

(482, 604, 3)


In [5]:
# plot_image(edges)
# print(edges.shape)


In [30]:
print(edges)
print(edges > 0.001)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.00048828 ... 0.         0.         0.        ]
 [0.         0.         0.00146484 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False  True ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [31]:
mask = edges > 0.001

edges[mask] = 1
edges[~mask] = 0

print(edges)
cann = cv2.Canny(edges, 100, 200)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/canny.cpp:829: error: (-215:Assertion failed) _src.depth() == CV_8U in function 'Canny'


In [ ]:
edges_trng = edge_detector.detectEdges(triangle)

In [ ]:
plot_image(edges_trng)

In [ ]:
import faiss

In [ ]:
index = faiss.IndexFlatL2(d)   # build the index


In [ ]:
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.


In [ ]:
print(xb.shape)

In [ ]:
index.add(edge_tensor)

In [ ]:
print(edge_tensor[0].numpy())